# TCN Classification with MR Dataset
<hr>

We will build a text classification model using TCN model on the Movie Review Dataset. Since there is no standard train/test split for this dataset, we will use 10-Fold Cross Validation (CV). 


## Load the library

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import random
# from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import KFold

%config IPCompleter.greedy=True
%config IPCompleter.use_jedi=False
# nltk.download('twitter_samples')

In [2]:
tf.config.list_physical_devices('GPU') 

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## Load the Dataset

In [3]:
corpus = pd.read_pickle('../../../0_data/SUBJ/SUBJ.pkl')
corpus.label = corpus.label.astype(int)
print(corpus.shape)
corpus

(10000, 3)


,sentence,label,split
0,"smart and alert , thirteen conversations about...",0,train
1,"color , musical bounce and warm seas lapping o...",0,train
2,it is not a mass market entertainment but an u...,0,train
3,a light hearted french film about the spiritua...,0,train
4,my wife is an actress has its moments in looki...,0,train
...,...,...,...
9995,"in the end , they discover that balance in lif...",1,train
9996,a counterfeit 1000 tomin bank note is passed i...,1,train
9997,enter the beautiful and mysterious secret agen...,1,train
9998,after listening to a missionary from china spe...,1,train


In [4]:
corpus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  10000 non-null  object
 1   label     10000 non-null  int32 
 2   split     10000 non-null  object
dtypes: int32(1), object(2)
memory usage: 195.4+ KB


In [5]:
corpus.groupby( by='label').count()

,sentence,split
label,,
0,5000,5000
1,5000,5000


In [6]:
# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

In [7]:
sentences[0]

'smart and alert , thirteen conversations about one thing is a small gem .'

<!--## Split Dataset-->

# Data Preprocessing
<hr>

Preparing data for word embedding, especially for pre-trained word embedding like Word2Vec or GloVe, __don't use standard preprocessing steps like stemming or stopword removal__. Compared to our approach on cleaning the text when doing word count based feature extraction (e.g. TFIDF) such as removing stopwords, stemming etc, now we will keep these words as we do not want to lose such information that might help the model learn better.

__Tomas Mikolov__, one of the developers of Word2Vec, in _word2vec-toolkit: google groups thread., 2015_, suggests only very minimal text cleaning is required when learning a word embedding model. Sometimes, it's good to disconnect
In short, what we will do is:
- Puntuations removal
- Lower the letter case
- Tokenization

The process above will be handled by __Tokenizer__ class in TensorFlow

- <b>One way to choose the maximum sequence length is to just pick the length of the longest sentence in the training set.</b>

In [8]:
# Define a function to compute the max length of sequence
def max_length(sequences):
    '''
    input:
        sequences: a 2D list of integer sequences
    output:
        max_length: the max length of the sequences
    '''
    max_length = 0
    for i, seq in enumerate(sequences):
        length = len(seq)
        if max_length < length:
            max_length = length
    return max_length

In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"

print("Example of sentence: ", sentences[4])

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)

# Turn the text into sequence
training_sequences = tokenizer.texts_to_sequences(sentences)
max_len = max_length(training_sequences)

print('Into a sequence of int:', training_sequences[4])

# Pad the sequence to have the same size
training_padded = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
print('Into a padded sequence:', training_padded[4])

Example of sentence:  my wife is an actress has its moments in looking at the comic effects of jealousy . in the end , though , it is only mildly amusing when it could have been so much more .
Into a sequence of int: [336, 208, 8, 16, 921, 25, 29, 312, 7, 313, 32, 2, 488, 551, 5, 3203, 7, 2, 129, 194, 10, 8, 60, 2330, 716, 39, 10, 128, 43, 82, 54, 81, 45]
Into a padded sequence: [ 336  208    8   16  921   25   29  312    7  313   32    2  488  551
    5 3203    7    2  129  194   10    8   60 2330  716   39   10  128
   43   82   54   81   45    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0]


In [10]:
word_index = tokenizer.word_index
# See the first 10 words in the vocabulary
for i, word in enumerate(word_index):
    print(word, word_index.get(word))
    if i==9:
        break
vocab_size = len(word_index)+1
print(vocab_size)

<UNK> 1
the 2
a 3
and 4
of 5
to 6
in 7
is 8
's 9
it 10
21324


# Model 1: Embedding Random
<hr>

## TCN Model

Now, we will build Temporal Convolutional Network (CNN) models to classify encoded documents as either positive or negative.

The model takes inspiration from https://github.com/philipperemy/keras-tcn and https://www.kaggle.com/christofhenkel/temporal-convolutional-network

__Arguments__
`TCN(nb_filters=64, kernel_size=2, nb_stacks=1, dilations=[1, 2, 4, 8, 16, 32], padding='causal', use_skip_connections=False, dropout_rate=0.0, return_sequences=True, activation='relu', kernel_initializer='he_normal', use_batch_norm=False, **kwargs)`

- `nb_filters`: Integer. The number of filters to use in the convolutional layers. Would be similar to units for LSTM.
- `kernel_size`: Integer. The size of the kernel to use in each convolutional layer.
- `dilations`: List. A dilation list. Example is: [1, 2, 4, 8, 16, 32, 64].
- `nb_stacks`: Integer. The number of stacks of residual blocks to use.
- `padding`: String. The padding to use in the convolutions. 'causal' for a causal network (as in the original implementation) and - `'same' for a non-causal network.
- `use_skip_connections`: Boolean. If we want to add skip connections from input to each residual block.
- `return_sequences`: Boolean. Whether to return the last output in the output sequence, or the full sequence.
- `dropout_rate`: Float between 0 and 1. Fraction of the input units to drop.
- `activation`: The activation used in the residual blocks o = activation(x + F(x)).
- `kernel_initializer`: Initializer for the kernel weights matrix (Conv1D).
- `use_batch_norm`: Whether to use batch normalization in the residual layers or not.
- `kwargs`: Any other arguments for configuring parent class Layer. For example "name=str", Name of the model. Use unique names when using multiple TCN.

Now, we will define our TCN model as follows:
- One TCN layer with 100 filters, kernel size 1-6, and relu and tanh activation function;
- Dropout size = 0.5;
- Optimizer: Adam (The best learning algorithm so far)
- Loss function: binary cross-entropy (suited for binary classification problem)

**Note**: 
- The whole purpose of dropout layers is to tackle the problem of over-fitting and to introduce generalization to the model. Hence it is advisable to keep dropout parameter near 0.5 in hidden layers. 
- https://missinglink.ai/guides/keras/keras-conv1d-working-1d-convolutional-neural-networks-keras/

In [11]:
from tcn import TCN, tcn_full_summary
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, SpatialDropout1D
from tensorflow.keras.layers import concatenate, GlobalAveragePooling1D, GlobalMaxPooling1D
from tensorflow.keras.models import Model

def define_model(kernel_size = 3, activation='relu', input_dim = None, output_dim=300, max_length = None ):
    
    inp = Input( shape=(max_length,))
    x = Embedding(input_dim=input_dim, output_dim=output_dim, input_length=max_length)(inp)
    x = SpatialDropout1D(0.1)(x)
    
    x = TCN(128,dilations = [1, 2, 4], return_sequences=True, activation = activation, name = 'tcn1')(x)
    x = TCN(64,dilations = [1, 2, 4], return_sequences=True, activation = activation, name = 'tcn2')(x)
    
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    
    conc = concatenate([avg_pool, max_pool])
    conc = Dense(16, activation="relu")(conc)
    conc = Dropout(0.1)(conc)
    outp = Dense(1, activation="sigmoid")(conc)    

    model = Model(inputs=inp, outputs=outp)
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    
    return model

In [12]:
model_0 = define_model( input_dim=1000, max_length=100)
model_0.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 100, 300)     300000      input_1[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d (SpatialDropo (None, 100, 300)     0           embedding[0][0]                  
__________________________________________________________________________________________________
tcn1 (TCN)                      (None, 100, 128)     279936      spatial_dropout1d[0][0]          
______________________________________________________________________________________________

In [13]:
# tcn_full_summary(model_0)

In [14]:
# class myCallback(tf.keras.callbacks.Callback):
#     # Overide the method on_epoch_end() for our benefit
#     def on_epoch_end(self, epoch, logs={}):
#         if (logs.get('accuracy') > 0.93):
#             print("\nReached 93% accuracy so cancelling training!")
#             self.model.stop_training=True


callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=6, verbose=2, 
                                             mode='auto', restore_best_weights=True)

## Train and Test the Model

In [15]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
activations = ['relu', 'tanh']
kernel_sizes = [1, 2, 3, 4, 5, 6]

columns = ['Activation', 'Filters', 'acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

exp = 0

for activation in activations:
    for kernel_size in kernel_sizes:
        # kfold.split() will return set indices for each split
        exp+=1
        print('-------------------------------------------')
        print('Training {}: {} activation, {} kernel size.'.format(exp, activation, kernel_size))
        print('-------------------------------------------')
        acc_list = []
        for train, test in kfold.split(sentences):
            
            train_x, test_x = [], []
            train_y, test_y = [], []
            
            for i in train:
                train_x.append(sentences[i])
                train_y.append(labels[i])

            for i in test:
                test_x.append(sentences[i])
                test_y.append(labels[i])

            # Turn the labels into a numpy array
            train_y = np.array(train_y)
            test_y = np.array(test_y)

            # encode data using
            # Cleaning and Tokenization
            tokenizer = Tokenizer(oov_token=oov_tok)
            tokenizer.fit_on_texts(train_x)

            # Turn the text into sequence
            training_sequences = tokenizer.texts_to_sequences(train_x)
            test_sequences = tokenizer.texts_to_sequences(test_x)

            max_len = max_length(training_sequences)

            # Pad the sequence to have the same size
            Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
            Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

            word_index = tokenizer.word_index
            vocab_size = len(word_index)+1

            # Define the input shape
            model = define_model(kernel_size, activation, input_dim=vocab_size, max_length=max_len)

            # Train the model and initialize test accuracy with 0
            acc = 0
            while(acc<0.6):
                
                model.fit(Xtrain, train_y, batch_size=50, epochs=100, verbose=1, 
                          callbacks=[callbacks], validation_data=(Xtest, test_y))

                # evaluate the model
                loss, acc = model.evaluate(Xtest, test_y, verbose=0)
                print('Test Accuracy: {}'.format(acc*100))
                
                if (acc<0.6):
                    print('The model suffered from local minimum. Retrain the model!')
                    model = define_model(kernel_size, activation, input_dim=vocab_size, max_length=max_len)
                    
                else:
                    print('Done!')

            acc_list.append(acc*100)
            
        mean_acc = np.array(acc_list).mean()
        parameters = [activation, kernel_size]
        entries = parameters + acc_list + [mean_acc]

        temp = pd.DataFrame([entries], columns=columns)
        record = record.append(temp, ignore_index=True)
        print()
        print(record)
        print()

-------------------------------------------
Training 1: relu activation, 1 kernel size.
-------------------------------------------


C:\Users\Diardano Raihan\Anaconda3\envs\tf-gpu\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass shuffle=True as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


Epoch 1/100
180/180 [==============================] - 49s 193ms/step - loss: 0.6279 - accuracy: 0.6093 - val_loss: 0.3478 - val_accuracy: 0.8760
Epoch 2/100
180/180 [==============================] - 33s 183ms/step - loss: 0.1678 - accuracy: 0.9438 - val_loss: 0.2591 - val_accuracy: 0.9000
Epoch 3/100
180/180 [==============================] - 33s 183ms/step - loss: 0.0407 - accuracy: 0.9871 - val_loss: 0.3496 - val_accuracy: 0.8860
Epoch 4/100
180/180 [==============================] - 33s 186ms/step - loss: 0.0168 - accuracy: 0.9963 - val_loss: 0.4255 - val_accuracy: 0.8900
Epoch 5/100
180/180 [==============================] - 37s 205ms/step - loss: 0.0139 - accuracy: 0.9968 - val_loss: 0.8022 - val_accuracy: 0.8740
Epoch 6/100
180/180 [==============================] - 40s 220ms/step - loss: 0.0129 - accuracy: 0.9961 - val_loss: 0.5538 - val_accuracy: 0.8890
Epoch 7/100
180/180 [==============================] - 38s 213ms/step - loss: 0.0084 - accuracy: 0.9973 - val_loss: 0.5221 -

180/180 [==============================] - 34s 191ms/step - loss: 0.0135 - accuracy: 0.9968 - val_loss: 0.3270 - val_accuracy: 0.9030
Epoch 5/100
180/180 [==============================] - 34s 188ms/step - loss: 0.0059 - accuracy: 0.9989 - val_loss: 0.3658 - val_accuracy: 0.9000
Epoch 6/100
180/180 [==============================] - 33s 186ms/step - loss: 0.0052 - accuracy: 0.9987 - val_loss: 0.6499 - val_accuracy: 0.9030
Epoch 7/100
180/180 [==============================] - 33s 186ms/step - loss: 0.0088 - accuracy: 0.9970 - val_loss: 0.6396 - val_accuracy: 0.9030
Epoch 8/100
180/180 [==============================] - 34s 186ms/step - loss: 0.0037 - accuracy: 0.9991 - val_loss: 0.7802 - val_accuracy: 0.8690
Epoch 9/100
180/180 [==============================] - 34s 187ms/step - loss: 0.0041 - accuracy: 0.9992 - val_loss: 0.8981 - val_accuracy: 0.8970
Restoring model weights from the end of the best epoch.
Epoch 00009: early stopping
Test Accuracy: 90.89999794960022
Done!
Epoch 1/100
1

Test Accuracy: 91.20000004768372
Done!
Epoch 1/100
180/180 [==============================] - 95s 502ms/step - loss: 0.6111 - accuracy: 0.6277 - val_loss: 0.2516 - val_accuracy: 0.9000
Epoch 2/100
180/180 [==============================] - 88s 492ms/step - loss: 0.1339 - accuracy: 0.9460 - val_loss: 0.2532 - val_accuracy: 0.9040
Epoch 3/100
180/180 [==============================] - 87s 482ms/step - loss: 0.0381 - accuracy: 0.9888 - val_loss: 0.5509 - val_accuracy: 0.8760
Epoch 4/100
180/180 [==============================] - 76s 424ms/step - loss: 0.0219 - accuracy: 0.9945 - val_loss: 0.6551 - val_accuracy: 0.8840
Epoch 5/100
180/180 [==============================] - 42s 234ms/step - loss: 0.0120 - accuracy: 0.9978 - val_loss: 0.7692 - val_accuracy: 0.8930
Epoch 6/100
180/180 [==============================] - 36s 200ms/step - loss: 0.0092 - accuracy: 0.9975 - val_loss: 0.6433 - val_accuracy: 0.8900
Epoch 7/100
180/180 [==============================] - 40s 219ms/step - loss: 0.0062 

...

180/180 [==============================] - 32s 180ms/step - loss: 0.0244 - accuracy: 0.9931 - val_loss: 0.4876 - val_accuracy: 0.8850
Epoch 9/100
180/180 [==============================] - 32s 180ms/step - loss: 0.0107 - accuracy: 0.9975 - val_loss: 0.5406 - val_accuracy: 0.8870
Epoch 10/100
180/180 [==============================] - 32s 180ms/step - loss: 0.0080 - accuracy: 0.9976 - val_loss: 0.6080 - val_accuracy: 0.8760
Epoch 11/100
180/180 [==============================] - 33s 181ms/step - loss: 0.0056 - accuracy: 0.9986 - val_loss: 0.5610 - val_accuracy: 0.8830
Epoch 12/100
180/180 [==============================] - 33s 181ms/step - loss: 0.0103 - accuracy: 0.9976 - val_loss: 0.5914 - val_accuracy: 0.8810
Restoring model weights from the end of the best epoch.
Epoch 00012: early stopping
Test Accuracy: 89.3999993801117
Done!
Epoch 1/100
180/180 [==============================] - 35s 176ms/step - loss: 0.5020 - accuracy: 0.7304 - val_loss: 0.2282 - val_accuracy: 0.9170
Epoch 2/100

Epoch 2/100
180/180 [==============================] - 27s 150ms/step - loss: 0.1084 - accuracy: 0.9619 - val_loss: 0.2572 - val_accuracy: 0.9050
Epoch 3/100
180/180 [==============================] - 28s 154ms/step - loss: 0.0382 - accuracy: 0.9888 - val_loss: 0.3158 - val_accuracy: 0.9040
Epoch 4/100
180/180 [==============================] - 28s 154ms/step - loss: 0.0168 - accuracy: 0.9951 - val_loss: 0.3746 - val_accuracy: 0.9040
Epoch 5/100
180/180 [==============================] - 28s 154ms/step - loss: 0.0149 - accuracy: 0.9962 - val_loss: 0.3895 - val_accuracy: 0.9140
Epoch 6/100
180/180 [==============================] - 28s 154ms/step - loss: 0.0075 - accuracy: 0.9981 - val_loss: 0.4321 - val_accuracy: 0.8980
Epoch 7/100
180/180 [==============================] - 28s 155ms/step - loss: 0.0105 - accuracy: 0.9967 - val_loss: 0.4486 - val_accuracy: 0.9080
Epoch 8/100
180/180 [==============================] - 28s 153ms/step - loss: 0.0071 - accuracy: 0.9981 - val_loss: 0.5232 -

180/180 [==============================] - 33s 181ms/step - loss: 0.0114 - accuracy: 0.9955 - val_loss: 0.4311 - val_accuracy: 0.8970
Restoring model weights from the end of the best epoch.
Epoch 00008: early stopping
Test Accuracy: 90.39999842643738
Done!
Epoch 1/100
180/180 [==============================] - 35s 176ms/step - loss: 0.5364 - accuracy: 0.7044 - val_loss: 0.2439 - val_accuracy: 0.8980
Epoch 2/100
180/180 [==============================] - 32s 177ms/step - loss: 0.1417 - accuracy: 0.9547 - val_loss: 0.2602 - val_accuracy: 0.9110
Epoch 3/100
180/180 [==============================] - 32s 180ms/step - loss: 0.0627 - accuracy: 0.9809 - val_loss: 0.2747 - val_accuracy: 0.9110
Epoch 4/100
180/180 [==============================] - 33s 181ms/step - loss: 0.0300 - accuracy: 0.9932 - val_loss: 0.3278 - val_accuracy: 0.9080
Epoch 5/100
180/180 [==============================] - 33s 182ms/step - loss: 0.0221 - accuracy: 0.9938 - val_loss: 0.4217 - val_accuracy: 0.8970
Epoch 6/100
1

Epoch 1/100
180/180 [==============================] - 35s 176ms/step - loss: 0.5508 - accuracy: 0.6834 - val_loss: 0.2464 - val_accuracy: 0.9000
Epoch 2/100
180/180 [==============================] - 32s 177ms/step - loss: 0.1304 - accuracy: 0.9573 - val_loss: 0.2481 - val_accuracy: 0.9020
Epoch 3/100
180/180 [==============================] - 33s 181ms/step - loss: 0.0528 - accuracy: 0.9837 - val_loss: 0.2971 - val_accuracy: 0.8870
Epoch 4/100
180/180 [==============================] - 33s 182ms/step - loss: 0.0240 - accuracy: 0.9945 - val_loss: 0.5146 - val_accuracy: 0.8770
Epoch 5/100
180/180 [==============================] - 33s 186ms/step - loss: 0.0236 - accuracy: 0.9942 - val_loss: 0.3744 - val_accuracy: 0.8820
Epoch 6/100
180/180 [==============================] - 33s 182ms/step - loss: 0.0280 - accuracy: 0.9921 - val_loss: 0.3789 - val_accuracy: 0.8820
Epoch 7/100
180/180 [==============================] - 33s 181ms/step - loss: 0.0224 - accuracy: 0.9939 - val_loss: 0.5561 -

180/180 [==============================] - 33s 181ms/step - loss: 0.0258 - accuracy: 0.9933 - val_loss: 0.3403 - val_accuracy: 0.9170
Epoch 5/100
180/180 [==============================] - 33s 182ms/step - loss: 0.0149 - accuracy: 0.9969 - val_loss: 0.4291 - val_accuracy: 0.9040
Epoch 6/100
180/180 [==============================] - 33s 182ms/step - loss: 0.0082 - accuracy: 0.9979 - val_loss: 0.4331 - val_accuracy: 0.8980
Epoch 7/100
180/180 [==============================] - 32s 180ms/step - loss: 0.0094 - accuracy: 0.9970 - val_loss: 0.5033 - val_accuracy: 0.8810
Epoch 8/100
180/180 [==============================] - 32s 180ms/step - loss: 0.0097 - accuracy: 0.9970 - val_loss: 0.5033 - val_accuracy: 0.8960
Epoch 9/100
180/180 [==============================] - 33s 181ms/step - loss: 0.0155 - accuracy: 0.9942 - val_loss: 0.4224 - val_accuracy: 0.8940
Epoch 10/100
180/180 [==============================] - 33s 181ms/step - loss: 0.0125 - accuracy: 0.9972 - val_loss: 0.4666 - val_accura

## Summary

In [16]:
record.sort_values(by='AVG', ascending=False)

,Activation,Filters,acc1,acc2,acc3,acc4,acc5,acc6,acc7,acc8,acc9,acc10,AVG
3,relu,4,91.600001,91.799998,90.600002,92.900002,89.899999,91.000003,90.700001,91.399997,92.799997,91.600001,91.430000
5,relu,6,92.799997,91.600001,91.000003,90.300000,90.600002,91.100001,93.400002,89.499998,92.199999,91.799998,91.430000
8,tanh,3,89.700001,91.200000,92.199999,91.700000,92.799997,92.199999,90.600002,91.200000,90.899998,91.399997,91.389999
1,relu,2,91.100001,91.200000,90.399998,91.900003,93.099999,91.600001,90.499997,90.700001,90.100002,92.799997,91.340000
9,tanh,4,90.600002,92.100000,92.199999,92.500001,90.100002,89.399999,92.600000,90.499997,90.399998,92.000002,91.240000
2,relu,3,90.700001,91.900003,91.299999,90.700001,91.700000,90.300000,93.099999,89.800000,89.700001,92.400002,91.160001
0,relu,1,89.999998,91.200000,91.600001,90.100002,91.399997,91.200000,90.899998,91.500002,91.700000,91.600001,91.120000
11,tanh,6,90.200001,91.299999,90.899998,88.900000,92.199999,91.700000,93.099999,91.000003,90.399998,91.399997,91.109999
4,relu,5,92.500001,90.899998,91.700000,91.000003,91.200000,90.700001,90.399998,89.800000,92.100000,90.700001,91.100000
7,tanh,2,91.900003,90.799999,89.999998,91.600001,90.399998,91.700000,91.900003,91.399997,90.399998,90.399998,91.050000


In [17]:
record[['Activation', 'AVG']].groupby(by='Activation').max().sort_values(by='AVG', ascending=False)

,AVG
Activation,
relu,91.430000
tanh,91.389999


In [18]:
report = record.sort_values(by='AVG', ascending=False)
report = report.to_excel('TCN_SUBJ.xlsx', sheet_name='random')

# Model 2: Word2Vec Static

__Using and updating pre-trained embeddings__
* In this part, we will create an Embedding layer in Tensorflow Keras using a pre-trained word embedding called Word2Vec 300-d tht has been trained 100 bilion words from Google News.
* In this part,  we will leave the embeddings fixed instead of updating them (dynamic).

1. __Load `Word2Vec` Pre-trained Word Embedding__

In [19]:
from gensim.models import KeyedVectors
word2vec = KeyedVectors.load_word2vec_format('../GoogleNews-vectors-negative300.bin', binary=True)

In [20]:
# Access the dense vector value for the word 'handsome'
# word2vec.word_vec('handsome') # 0.11376953
word2vec.word_vec('cool') # 1.64062500e-01

array([ 1.64062500e-01,  1.87500000e-01, -4.10156250e-02,  1.25000000e-01,
       -3.22265625e-02,  8.69140625e-02,  1.19140625e-01, -1.26953125e-01,
        1.77001953e-02,  8.83789062e-02,  2.12402344e-02, -2.00195312e-01,
        4.83398438e-02, -1.01074219e-01, -1.89453125e-01,  2.30712891e-02,
        1.17675781e-01,  7.51953125e-02, -8.39843750e-02, -1.33666992e-02,
        1.53320312e-01,  4.08203125e-01,  3.80859375e-02,  3.36914062e-02,
       -4.02832031e-02, -6.88476562e-02,  9.03320312e-02,  2.12890625e-01,
        1.72119141e-02, -6.44531250e-02, -1.29882812e-01,  1.40625000e-01,
        2.38281250e-01,  1.37695312e-01, -1.76757812e-01, -2.71484375e-01,
       -1.36718750e-01, -1.69921875e-01, -9.15527344e-03,  3.47656250e-01,
        2.22656250e-01, -3.06640625e-01,  1.98242188e-01,  1.33789062e-01,
       -4.34570312e-02, -5.12695312e-02, -3.46679688e-02, -8.49609375e-02,
        1.01562500e-01,  1.42578125e-01, -7.95898438e-02,  1.78710938e-01,
        2.30468750e-01,  

2. __Check number of training words present in Word2Vec__

In [21]:
def training_words_in_word2vector(word_to_vec_map, word_to_index):
    '''
    input:
        word_to_vec_map: a word2vec GoogleNews-vectors-negative300.bin model loaded using gensim.models
        word_to_index: word to index mapping from training set
    '''
    
    vocab_size = len(word_to_index) + 1
    count = 0
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            count+=1
            
    return print('Found {} words present from {} training vocabulary in the set of pre-trained word vector'.format(count, vocab_size))

In [22]:
# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(sentences)

word_index = tokenizer.word_index
training_words_in_word2vector(word2vec, word_index)

Found 17913 words present from 21324 training vocabulary in the set of pre-trained word vector


2. __Define a `pretrained_embedding_layer` function__

In [23]:
emb_mean = word2vec.vectors.mean()
emb_std = word2vec.vectors.std()

In [24]:
from tensorflow.keras.layers import Embedding

def pretrained_embedding_matrix(word_to_vec_map, word_to_index, emb_mean, emb_std):
    '''
    input:
        word_to_vec_map: a word2vec GoogleNews-vectors-negative300.bin model loaded using gensim.models
        word_to_index: word to index mapping from training set
    '''
    np.random.seed(2021)
    
    # adding 1 to fit Keras embedding (requirement)
    vocab_size = len(word_to_index) + 1
    # define dimensionality of your pre-trained word vectors (= 300)
    emb_dim = word_to_vec_map.word_vec('handsome').shape[0]
    
    # initialize the matrix with generic normal distribution values
    embed_matrix = np.random.normal(emb_mean, emb_std, (vocab_size, emb_dim))
    
    # Set each row "idx" of the embedding matrix to be 
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        if word in word_to_vec_map:
            embed_matrix[idx] = word_to_vec_map.get_vector(word)
            
    return embed_matrix

In [25]:
# Test the function
w_2_i = {'<UNK>': 1, 'handsome': 2, 'cool': 3, 'shit': 4 }
em_matrix = pretrained_embedding_matrix(word2vec, w_2_i, emb_mean, emb_std)
em_matrix

array([[ 0.19468211,  0.08648376, -0.05924511, ..., -0.16683994,
        -0.09975549, -0.08595189],
       [-0.13509196, -0.07441947,  0.15388953, ..., -0.05400787,
        -0.13156594, -0.05996158],
       [ 0.11376953,  0.1796875 , -0.265625  , ..., -0.21875   ,
        -0.03930664,  0.20996094],
       [ 0.1640625 ,  0.1875    , -0.04101562, ...,  0.10888672,
        -0.01019287,  0.02075195],
       [ 0.10888672, -0.16699219,  0.08984375, ..., -0.19628906,
        -0.23144531,  0.04614258]])

## TCN Model

In [26]:
def define_model_2(kernel_size = 3, activation='relu', input_dim = None, 
                   output_dim=300, max_length = None, emb_matrix = None):
    
    inp = Input( shape=(max_length,))
    x = Embedding(input_dim=input_dim, 
                  output_dim=output_dim, 
                  input_length=max_length,
                  # Assign the embedding weight with word2vec embedding marix
                  weights = [emb_matrix],
                  # Set the weight to be not trainable (static)
                  trainable = False)(inp)
    
    x = SpatialDropout1D(0.1)(x)
    
    x = TCN(128,dilations = [1, 2, 4], return_sequences=True, activation = activation, name = 'tcn1')(x)
    x = TCN(64,dilations = [1, 2, 4], return_sequences=True, activation = activation, name = 'tcn2')(x)
    
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    
    conc = concatenate([avg_pool, max_pool])
    conc = Dense(16, activation="relu")(conc)
    conc = Dropout(0.1)(conc)
    outp = Dense(1, activation="sigmoid")(conc)    

    model = Model(inputs=inp, outputs=outp)
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    
    return model

In [27]:
model_0 = define_model_2( input_dim=1000, max_length=100, emb_matrix=np.random.rand(1000, 300))
model_0.summary()

Model: "model_124"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_125 (InputLayer)          [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding_124 (Embedding)       (None, 100, 300)     300000      input_125[0][0]                  
__________________________________________________________________________________________________
spatial_dropout1d_124 (SpatialD (None, 100, 300)     0           embedding_124[0][0]              
__________________________________________________________________________________________________
tcn1 (TCN)                      (None, 100, 128)     279936      spatial_dropout1d_124[0][0]      
__________________________________________________________________________________________

## Train and Test the Model

In [28]:
# class myCallback(tf.keras.callbacks.Callback):
#     # Overide the method on_epoch_end() for our benefit
#     def on_epoch_end(self, epoch, logs={}):
#         if (logs.get('accuracy') >= 0.9):
#             print("\nReached 90% accuracy so cancelling training!")
#             self.model.stop_training=True

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=6, verbose=2, 
                                             mode='auto', restore_best_weights=True)

In [29]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
activations = ['relu']
print('Loading embedding statistics . . .')
emb_mean = emb_mean
emb_std = emb_std
print('Done!')
kernel_sizes = [1, 2, 3, 4, 5, 6]

columns = ['Activation', 'Filters', 'acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record2 = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)
exp = 0

for activation in activations:
    for kernel_size in kernel_sizes:
        exp+=1
        print('-------------------------------------------')
        print('Training {}: {} activation, {} kernel size.'.format(exp, activation, kernel_size))
        print('-------------------------------------------')
        
        # kfold.split() will return set indices for each split
        acc_list = []
        for train, test in kfold.split(sentences):
            
            train_x, test_x = [], []
            train_y, test_y = [], []
            
            for i in train:
                train_x.append(sentences[i])
                train_y.append(labels[i])

            for i in test:
                test_x.append(sentences[i])
                test_y.append(labels[i])

            # Turn the labels into a numpy array
            train_y = np.array(train_y)
            test_y = np.array(test_y)

            # encode data using
            # Cleaning and Tokenization
            tokenizer = Tokenizer(oov_token=oov_tok)
            tokenizer.fit_on_texts(train_x)

            # Turn the text into sequence
            training_sequences = tokenizer.texts_to_sequences(train_x)
            test_sequences = tokenizer.texts_to_sequences(test_x)

            max_len = max_length(training_sequences)

            # Pad the sequence to have the same size
            Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
            Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

            word_index = tokenizer.word_index
            vocab_size = len(word_index)+1
            
            
            emb_matrix = pretrained_embedding_matrix(word2vec, word_index, emb_mean, emb_std)
            
            # Define the input shape
            model = define_model_2(kernel_size, activation, input_dim=vocab_size, 
                                 max_length=max_len, emb_matrix=emb_matrix)

            # Train the model and initialize test accuracy with 0
            acc = 0
            while(acc<0.6):
                
                model.fit(Xtrain, train_y, batch_size=50, epochs=100, verbose=1, 
                          callbacks=[callbacks], validation_data=(Xtest, test_y))

                # evaluate the model
                loss, acc = model.evaluate(Xtest, test_y, verbose=0)
                print('Test Accuracy: {}'.format(acc*100))
                
                if (acc<0.6):
                    print('The model suffered from local minimum. Retrain the model!')
                    model = define_model_2(kernel_size, activation, input_dim=vocab_size, 
                                           max_length=max_len, emb_matrix=emb_matrix)
                else:
                    print('Done!')
                    
            acc_list.append(acc*100)
            
        mean_acc = np.array(acc_list).mean()
        parameters = [activation, kernel_size]
        entries = parameters + acc_list + [mean_acc]

        temp = pd.DataFrame([entries], columns=columns)
        record2 = record2.append(temp, ignore_index=True)
        print()
        print(record2)
        print()

Loading embedding statistics . . .
Done!
-------------------------------------------
Training 1: relu activation, 1 kernel size.
-------------------------------------------


C:\Users\Diardano Raihan\Anaconda3\envs\tf-gpu\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass shuffle=True as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


Epoch 1/100
180/180 [==============================] - 24s 111ms/step - loss: 0.5742 - accuracy: 0.6837 - val_loss: 0.2776 - val_accuracy: 0.9020
Epoch 2/100
180/180 [==============================] - 19s 105ms/step - loss: 0.2541 - accuracy: 0.9006 - val_loss: 0.2295 - val_accuracy: 0.9010
Epoch 3/100
180/180 [==============================] - 19s 104ms/step - loss: 0.2098 - accuracy: 0.9195 - val_loss: 0.2224 - val_accuracy: 0.9130
Epoch 4/100
180/180 [==============================] - 19s 104ms/step - loss: 0.1756 - accuracy: 0.9323 - val_loss: 0.2340 - val_accuracy: 0.9150
Epoch 5/100
180/180 [==============================] - 19s 105ms/step - loss: 0.1500 - accuracy: 0.9471 - val_loss: 0.1974 - val_accuracy: 0.9250
Epoch 6/100
180/180 [==============================] - 19s 104ms/step - loss: 0.1130 - accuracy: 0.9564 - val_loss: 0.2179 - val_accuracy: 0.9230
Epoch 7/100
180/180 [==============================] - 19s 105ms/step - loss: 0.0994 - accuracy: 0.9687 - val_loss: 0.1966 -

180/180 [==============================] - 21s 115ms/step - loss: 0.1407 - accuracy: 0.9469 - val_loss: 0.2370 - val_accuracy: 0.9110
Epoch 6/100
180/180 [==============================] - 21s 115ms/step - loss: 0.1072 - accuracy: 0.9603 - val_loss: 0.2098 - val_accuracy: 0.9150
Epoch 7/100
180/180 [==============================] - 21s 114ms/step - loss: 0.1021 - accuracy: 0.9632 - val_loss: 0.2360 - val_accuracy: 0.9250
Epoch 8/100
180/180 [==============================] - 21s 115ms/step - loss: 0.0653 - accuracy: 0.9773 - val_loss: 0.2727 - val_accuracy: 0.9250
Epoch 9/100
180/180 [==============================] - 21s 115ms/step - loss: 0.0525 - accuracy: 0.9819 - val_loss: 0.3040 - val_accuracy: 0.9190
Epoch 10/100
180/180 [==============================] - 21s 115ms/step - loss: 0.0516 - accuracy: 0.9819 - val_loss: 0.2986 - val_accuracy: 0.8910
Epoch 11/100
180/180 [==============================] - 21s 115ms/step - loss: 0.0667 - accuracy: 0.9776 - val_loss: 0.3511 - val_accur

180/180 [==============================] - 20s 113ms/step - loss: 0.1260 - accuracy: 0.9545 - val_loss: 0.2608 - val_accuracy: 0.9050
Epoch 7/100
180/180 [==============================] - 21s 115ms/step - loss: 0.0930 - accuracy: 0.9666 - val_loss: 0.3048 - val_accuracy: 0.8910
Epoch 8/100
180/180 [==============================] - 23s 130ms/step - loss: 0.0770 - accuracy: 0.9741 - val_loss: 0.3407 - val_accuracy: 0.8890
Epoch 9/100
180/180 [==============================] - 21s 115ms/step - loss: 0.0514 - accuracy: 0.9826 - val_loss: 0.3855 - val_accuracy: 0.9090
Epoch 10/100
180/180 [==============================] - 21s 114ms/step - loss: 0.0660 - accuracy: 0.9763 - val_loss: 0.3282 - val_accuracy: 0.9170
Epoch 11/100
180/180 [==============================] - 21s 115ms/step - loss: 0.0374 - accuracy: 0.9875 - val_loss: 0.4026 - val_accuracy: 0.9070
Epoch 12/100
180/180 [==============================] - 21s 114ms/step - loss: 0.0418 - accuracy: 0.9846 - val_loss: 0.3390 - val_accu

...

Epoch 7/100
180/180 [==============================] - 21s 115ms/step - loss: 0.1133 - accuracy: 0.9576 - val_loss: 0.2413 - val_accuracy: 0.9130
Epoch 8/100
180/180 [==============================] - 21s 115ms/step - loss: 0.0759 - accuracy: 0.9762 - val_loss: 0.2732 - val_accuracy: 0.9170
Epoch 9/100
180/180 [==============================] - 21s 115ms/step - loss: 0.0548 - accuracy: 0.9822 - val_loss: 0.2683 - val_accuracy: 0.9050
Epoch 10/100
180/180 [==============================] - 21s 115ms/step - loss: 0.0499 - accuracy: 0.9837 - val_loss: 0.3143 - val_accuracy: 0.9060
Epoch 11/100
180/180 [==============================] - 21s 115ms/step - loss: 0.0574 - accuracy: 0.9811 - val_loss: 0.2848 - val_accuracy: 0.9150
Epoch 12/100
180/180 [==============================] - 21s 115ms/step - loss: 0.0368 - accuracy: 0.9880 - val_loss: 0.4157 - val_accuracy: 0.9060
Epoch 13/100
180/180 [==============================] - 24s 131ms/step - loss: 0.0301 - accuracy: 0.9898 - val_loss: 0.38

180/180 [==============================] - 21s 114ms/step - loss: 0.1010 - accuracy: 0.9632 - val_loss: 0.2224 - val_accuracy: 0.9220
Epoch 8/100
180/180 [==============================] - 21s 114ms/step - loss: 0.0872 - accuracy: 0.9693 - val_loss: 0.2250 - val_accuracy: 0.9280
Epoch 9/100
180/180 [==============================] - 21s 114ms/step - loss: 0.0759 - accuracy: 0.9718 - val_loss: 0.2791 - val_accuracy: 0.9230
Epoch 10/100
180/180 [==============================] - 21s 114ms/step - loss: 0.0605 - accuracy: 0.9786 - val_loss: 0.2463 - val_accuracy: 0.9280
Epoch 11/100
180/180 [==============================] - 21s 115ms/step - loss: 0.0519 - accuracy: 0.9793 - val_loss: 0.3002 - val_accuracy: 0.9370
Epoch 12/100
180/180 [==============================] - 21s 114ms/step - loss: 0.0412 - accuracy: 0.9841 - val_loss: 0.2676 - val_accuracy: 0.9250
Epoch 13/100
180/180 [==============================] - 21s 114ms/step - loss: 0.0386 - accuracy: 0.9841 - val_loss: 0.2578 - val_acc

180/180 [==============================] - 20s 113ms/step - loss: 0.6932 - accuracy: 0.5047 - val_loss: 0.6932 - val_accuracy: 0.4960
Epoch 7/100
180/180 [==============================] - 20s 114ms/step - loss: 0.6932 - accuracy: 0.5043 - val_loss: 0.6932 - val_accuracy: 0.4960
Restoring model weights from the end of the best epoch.
Epoch 00007: early stopping
Test Accuracy: 49.59999918937683
The model suffered from local minimum. Retrain the model!
Epoch 1/100
180/180 [==============================] - 24s 115ms/step - loss: 0.7005 - accuracy: 0.6354 - val_loss: 0.2775 - val_accuracy: 0.8960
Epoch 2/100
180/180 [==============================] - 20s 113ms/step - loss: 0.2641 - accuracy: 0.8976 - val_loss: 0.2561 - val_accuracy: 0.8990
Epoch 3/100
180/180 [==============================] - 20s 114ms/step - loss: 0.2017 - accuracy: 0.9227 - val_loss: 0.2338 - val_accuracy: 0.9050
Epoch 4/100
180/180 [==============================] - 20s 114ms/step - loss: 0.1643 - accuracy: 0.9388 - v

180/180 [==============================] - 21s 114ms/step - loss: 0.0412 - accuracy: 0.9872 - val_loss: 0.2997 - val_accuracy: 0.9070
Restoring model weights from the end of the best epoch.
Epoch 00011: early stopping
Test Accuracy: 91.69999957084656
Done!
Epoch 1/100
180/180 [==============================] - 24s 114ms/step - loss: 0.5703 - accuracy: 0.6923 - val_loss: 0.2539 - val_accuracy: 0.9030
Epoch 2/100
180/180 [==============================] - 20s 113ms/step - loss: 0.2553 - accuracy: 0.8938 - val_loss: 0.2200 - val_accuracy: 0.9110
Epoch 3/100
180/180 [==============================] - 21s 115ms/step - loss: 0.2100 - accuracy: 0.9156 - val_loss: 0.2153 - val_accuracy: 0.9160
Epoch 4/100
180/180 [==============================] - 23s 127ms/step - loss: 0.1745 - accuracy: 0.9361 - val_loss: 0.2387 - val_accuracy: 0.9070
Epoch 5/100
180/180 [==============================] - 21s 115ms/step - loss: 0.1479 - accuracy: 0.9415 - val_loss: 0.2021 - val_accuracy: 0.9260
Epoch 6/100
1

Test Accuracy: 92.29999780654907
Done!
Epoch 1/100
180/180 [==============================] - 26s 112ms/step - loss: 0.6335 - accuracy: 0.6313 - val_loss: 0.2739 - val_accuracy: 0.8960
Epoch 2/100
180/180 [==============================] - 20s 111ms/step - loss: 0.2620 - accuracy: 0.8986 - val_loss: 0.2496 - val_accuracy: 0.9020
Epoch 3/100
180/180 [==============================] - 20s 112ms/step - loss: 0.2077 - accuracy: 0.9172 - val_loss: 0.2335 - val_accuracy: 0.9080
Epoch 4/100
180/180 [==============================] - 20s 113ms/step - loss: 0.1748 - accuracy: 0.9329 - val_loss: 0.2223 - val_accuracy: 0.9110
Epoch 5/100
180/180 [==============================] - 20s 114ms/step - loss: 0.1478 - accuracy: 0.9486 - val_loss: 0.2409 - val_accuracy: 0.9020
Epoch 6/100
180/180 [==============================] - 20s 114ms/step - loss: 0.1295 - accuracy: 0.9531 - val_loss: 0.3307 - val_accuracy: 0.8750
Epoch 7/100
180/180 [==============================] - 20s 114ms/step - loss: 0.0951 

## Summary

In [30]:
record2.sort_values(by='AVG', ascending=False)

,Activation,Filters,acc1,acc2,acc3,acc4,acc5,acc6,acc7,acc8,acc9,acc10,AVG
0,relu,1,93.500000,91.900003,91.299999,91.700000,92.500001,91.799998,91.799998,93.500000,91.700000,93.699998,92.340000
2,relu,3,91.299999,91.600001,94.300002,92.900002,92.400002,92.500001,91.200000,92.100000,92.299998,92.000002,92.260001
4,relu,5,92.600000,92.199999,91.600001,91.700000,91.799998,93.599999,90.799999,93.699998,92.100000,92.199999,92.229999
3,relu,4,92.100000,92.900002,91.700000,92.699999,93.199998,92.900002,92.799997,91.100001,90.899998,91.100001,92.140000
5,relu,6,91.600001,92.900002,91.700000,92.600000,91.600001,92.299998,91.299999,92.299998,90.899998,93.300003,92.050000
1,relu,2,92.199999,91.399997,92.600000,92.600000,91.900003,92.400002,90.700001,92.600000,92.400002,91.600001,92.040001


In [31]:
record2[['Activation', 'AVG']].groupby(by='Activation').max().sort_values(by='AVG', ascending=False)

,AVG
Activation,
relu,92.34


In [32]:
report = record2.sort_values(by='AVG', ascending=False)
report = report.to_excel('TCN_SUBJ_2.xlsx', sheet_name='static')

# Model 3: Word2Vec - Dynamic

* In this part,  we will fine tune the embeddings while training (dynamic).

## CNN Model

In [33]:
def define_model_3(kernel_size = 3, activation='relu', input_dim = None, 
                   output_dim=300, max_length = None, emb_matrix = None):
    
    inp = Input( shape=(max_length,))
    x = Embedding(input_dim=input_dim, 
                  output_dim=output_dim, 
                  input_length=max_length,
                  # Assign the embedding weight with word2vec embedding marix
                  weights = [emb_matrix],
                  # Set the weight to be not trainable (static)
                  trainable = True)(inp)
    
    x = SpatialDropout1D(0.1)(x)
    
    x = TCN(128,dilations = [1, 2, 4], return_sequences=True, activation = activation, name = 'tcn1')(x)
    x = TCN(64,dilations = [1, 2, 4], return_sequences=True, activation = activation, name = 'tcn2')(x)
    
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    
    conc = concatenate([avg_pool, max_pool])
    conc = Dense(16, activation="relu")(conc)
    conc = Dropout(0.1)(conc)
    outp = Dense(1, activation="sigmoid")(conc)    

    model = Model(inputs=inp, outputs=outp)
    model.compile( loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    
    return model

In [34]:
model_0 = define_model_3( input_dim=1000, max_length=100, emb_matrix=np.random.rand(1000, 300))
model_0.summary()

Model: "model_187"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_188 (InputLayer)          [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding_187 (Embedding)       (None, 100, 300)     300000      input_188[0][0]                  
__________________________________________________________________________________________________
spatial_dropout1d_187 (SpatialD (None, 100, 300)     0           embedding_187[0][0]              
__________________________________________________________________________________________________
tcn1 (TCN)                      (None, 100, 128)     279936      spatial_dropout1d_187[0][0]      
__________________________________________________________________________________________

## Train and Test the Model

In [35]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') > 0.93):
            print("\nReached 93% accuracy so cancelling training!")
            self.model.stop_training=True

callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=6, verbose=2, 
                                             mode='auto', restore_best_weights=True)

In [36]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"
activations = ['relu']
print('Loading embedding statistics . . .')
emb_mean = emb_mean
emb_std = emb_std
print('Done!')
kernel_sizes = [1, 2, 3, 4, 5, 6]

columns = ['Activation', 'Filters', 'acc1', 'acc2', 'acc3', 'acc4', 'acc5', 'acc6', 'acc7', 'acc8', 'acc9', 'acc10', 'AVG']
record3 = pd.DataFrame(columns = columns)

# prepare cross validation with 10 splits and shuffle = True
kfold = KFold(10, True)

# Separate the sentences and the labels
sentences, labels = list(corpus.sentence), list(corpus.label)

exp = 0

for activation in activations:
    for kernel_size in kernel_sizes:
        exp+=1
        print('-------------------------------------------')
        print('Training {}: {} activation, {} kernel size.'.format(exp, activation, kernel_size))
        print('-------------------------------------------')
        
        # kfold.split() will return set indices for each split
        acc_list = []
        for train, test in kfold.split(sentences):
            
            train_x, test_x = [], []
            train_y, test_y = [], []
            
            for i in train:
                train_x.append(sentences[i])
                train_y.append(labels[i])

            for i in test:
                test_x.append(sentences[i])
                test_y.append(labels[i])

            # Turn the labels into a numpy array
            train_y = np.array(train_y)
            test_y = np.array(test_y)

            # encode data using
            # Cleaning and Tokenization
            tokenizer = Tokenizer(oov_token=oov_tok)
            tokenizer.fit_on_texts(train_x)

            # Turn the text into sequence
            training_sequences = tokenizer.texts_to_sequences(train_x)
            test_sequences = tokenizer.texts_to_sequences(test_x)

            max_len = max_length(training_sequences)

            # Pad the sequence to have the same size
            Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
            Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

            word_index = tokenizer.word_index
            vocab_size = len(word_index)+1
            
            
            emb_matrix = pretrained_embedding_matrix(word2vec, word_index, emb_mean, emb_std)
            
            # Define the input shape
            model = define_model_3(kernel_size, activation, input_dim=vocab_size, 
                                 max_length=max_len, emb_matrix=emb_matrix)

            # Train the model and initialize test accuracy with 0
            acc = 0
            while(acc<0.6):
                
                model.fit(Xtrain, train_y, batch_size=50, epochs=100, verbose=1, 
                          callbacks=[callbacks], validation_data=(Xtest, test_y))

                # evaluate the model
                loss, acc = model.evaluate(Xtest, test_y, verbose=0)
                print('Test Accuracy: {}'.format(acc*100))
                
                if (acc<0.6):
                    print('The model suffered from local minimum. Retrain the model!')
                    model = define_model_3(kernel_size, activation, input_dim=vocab_size, 
                                           max_length=max_len, emb_matrix=emb_matrix)
                else:
                    print('Done!')
                    
            acc_list.append(acc*100)
            
        mean_acc = np.array(acc_list).mean()
        parameters = [activation, kernel_size]
        entries = parameters + acc_list + [mean_acc]

        temp = pd.DataFrame([entries], columns=columns)
        record3 = record3.append(temp, ignore_index=True)
        print()
        print(record3)
        print()

Loading embedding statistics . . .
Done!
-------------------------------------------
Training 1: relu activation, 1 kernel size.
-------------------------------------------


C:\Users\Diardano Raihan\Anaconda3\envs\tf-gpu\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass shuffle=True as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


Epoch 1/100
180/180 [==============================] - 35s 175ms/step - loss: 0.5823 - accuracy: 0.6490 - val_loss: 0.2257 - val_accuracy: 0.9020
Epoch 2/100
180/180 [==============================] - 32s 175ms/step - loss: 0.1577 - accuracy: 0.9424 - val_loss: 0.2051 - val_accuracy: 0.9210
Epoch 3/100
180/180 [==============================] - 32s 178ms/step - loss: 0.0719 - accuracy: 0.9766 - val_loss: 0.2442 - val_accuracy: 0.9220
Epoch 4/100
180/180 [==============================] - 32s 179ms/step - loss: 0.0309 - accuracy: 0.9912 - val_loss: 0.4831 - val_accuracy: 0.9020
Epoch 5/100
180/180 [==============================] - 32s 178ms/step - loss: 0.0252 - accuracy: 0.9935 - val_loss: 0.3253 - val_accuracy: 0.9140
Epoch 6/100
180/180 [==============================] - 32s 179ms/step - loss: 0.0100 - accuracy: 0.9979 - val_loss: 0.5049 - val_accuracy: 0.9100
Epoch 7/100
180/180 [==============================] - 32s 178ms/step - loss: 0.0091 - accuracy: 0.9981 - val_loss: 0.6313 -

Epoch 3/100
180/180 [==============================] - 28s 153ms/step - loss: 0.6931 - accuracy: 0.5086 - val_loss: 0.6935 - val_accuracy: 0.4680
Epoch 4/100
180/180 [==============================] - 28s 156ms/step - loss: 0.6933 - accuracy: 0.4850 - val_loss: 0.6936 - val_accuracy: 0.4680
Epoch 5/100
180/180 [==============================] - 28s 156ms/step - loss: 0.6930 - accuracy: 0.5124 - val_loss: 0.6935 - val_accuracy: 0.4680
Epoch 6/100
180/180 [==============================] - 28s 156ms/step - loss: 0.6931 - accuracy: 0.5107 - val_loss: 0.6935 - val_accuracy: 0.4680
Epoch 7/100
180/180 [==============================] - 28s 155ms/step - loss: 0.6932 - accuracy: 0.4985 - val_loss: 0.6937 - val_accuracy: 0.4680
Restoring model weights from the end of the best epoch.
Epoch 00007: early stopping
Test Accuracy: 46.799999475479126
The model suffered from local minimum. Retrain the model!
Epoch 1/100
180/180 [==============================] - 32s 156ms/step - loss: 0.7195 - accurac

180/180 [==============================] - 33s 181ms/step - loss: 0.0257 - accuracy: 0.9926 - val_loss: 0.4064 - val_accuracy: 0.8920
Epoch 5/100
180/180 [==============================] - 33s 181ms/step - loss: 0.0170 - accuracy: 0.9938 - val_loss: 0.4465 - val_accuracy: 0.9010
Epoch 6/100
180/180 [==============================] - 32s 180ms/step - loss: 0.0087 - accuracy: 0.9971 - val_loss: 0.4198 - val_accuracy: 0.9110
Epoch 7/100
180/180 [==============================] - 32s 181ms/step - loss: 0.0090 - accuracy: 0.9974 - val_loss: 0.4791 - val_accuracy: 0.9100
Epoch 8/100
180/180 [==============================] - 33s 181ms/step - loss: 0.0109 - accuracy: 0.9956 - val_loss: 0.6345 - val_accuracy: 0.8940
Restoring model weights from the end of the best epoch.
Epoch 00008: early stopping
Test Accuracy: 92.1999990940094
Done!

  Activation Filters       acc1       acc2       acc3       acc4       acc5  \
0       relu       1  92.199999  91.299999  92.400002  92.199999  92.400002   



...

Restoring model weights from the end of the best epoch.
Epoch 00009: early stopping
Test Accuracy: 91.69999957084656
Done!
Epoch 1/100
180/180 [==============================] - 35s 176ms/step - loss: 0.6402 - accuracy: 0.6898 - val_loss: 0.2571 - val_accuracy: 0.8880
Epoch 2/100
180/180 [==============================] - 31s 172ms/step - loss: 0.1547 - accuracy: 0.9416 - val_loss: 0.1780 - val_accuracy: 0.9340
Epoch 3/100
180/180 [==============================] - 31s 173ms/step - loss: 0.0698 - accuracy: 0.9790 - val_loss: 0.2743 - val_accuracy: 0.9000
Epoch 4/100
180/180 [==============================] - 31s 173ms/step - loss: 0.0383 - accuracy: 0.9891 - val_loss: 0.2465 - val_accuracy: 0.9260
Epoch 5/100
180/180 [==============================] - 31s 174ms/step - loss: 0.0149 - accuracy: 0.9966 - val_loss: 0.3295 - val_accuracy: 0.9160
Epoch 6/100
180/180 [==============================] - 31s 173ms/step - loss: 0.0157 - accuracy: 0.9963 - val_loss: 0.3918 - val_accuracy: 0.9300
E

Epoch 9/100
180/180 [==============================] - 32s 177ms/step - loss: 0.0357 - accuracy: 0.9905 - val_loss: 0.4375 - val_accuracy: 0.9100
Restoring model weights from the end of the best epoch.
Epoch 00009: early stopping
Test Accuracy: 92.90000200271606
Done!
Epoch 1/100
180/180 [==============================] - 37s 186ms/step - loss: 0.7279 - accuracy: 0.6633 - val_loss: 0.2299 - val_accuracy: 0.9180
Epoch 2/100
180/180 [==============================] - 35s 194ms/step - loss: 0.1997 - accuracy: 0.9240 - val_loss: 0.2425 - val_accuracy: 0.9100
Epoch 3/100
180/180 [==============================] - 34s 190ms/step - loss: 0.0701 - accuracy: 0.9763 - val_loss: 0.3698 - val_accuracy: 0.8780
Epoch 4/100
180/180 [==============================] - 34s 187ms/step - loss: 0.0244 - accuracy: 0.9918 - val_loss: 0.3557 - val_accuracy: 0.9150
Epoch 5/100
180/180 [==============================] - 34s 189ms/step - loss: 0.0159 - accuracy: 0.9943 - val_loss: 0.4438 - val_accuracy: 0.9160
E

180/180 [==============================] - 32s 180ms/step - loss: 0.0059 - accuracy: 0.9990 - val_loss: 0.6948 - val_accuracy: 0.9050
Restoring model weights from the end of the best epoch.
Epoch 00009: early stopping
Test Accuracy: 91.90000295639038
Done!

  Activation Filters       acc1       acc2       acc3       acc4       acc5  \
0       relu       1  92.199999  91.299999  92.400002  92.199999  92.400002   
1       relu       2  91.900003  91.799998  91.200000  92.400002  92.900002   
2       relu       3  92.900002  93.900001  92.400002  91.399997  92.500001   
3       relu       4  92.699999  93.599999  91.799998  92.000002  92.000002   
4       relu       5  91.299999  91.700000  93.099999  92.900002  91.799998   

        acc6       acc7       acc8       acc9      acc10        AVG  
0  92.199999  91.799998  91.700000  92.100000  92.199999  92.050000  
1  91.200000  92.299998  91.299999  93.300003  91.399997  91.970000  
2  92.199999  92.000002  92.600000  92.100000  92.000002 

Epoch 7/100
180/180 [==============================] - 29s 159ms/step - loss: 0.0057 - accuracy: 0.9982 - val_loss: 0.4947 - val_accuracy: 0.9090
Epoch 8/100
180/180 [==============================] - 29s 159ms/step - loss: 0.0041 - accuracy: 0.9994 - val_loss: 0.5698 - val_accuracy: 0.8870
Epoch 9/100
180/180 [==============================] - 29s 159ms/step - loss: 0.0139 - accuracy: 0.9955 - val_loss: 0.4725 - val_accuracy: 0.8980
Restoring model weights from the end of the best epoch.
Epoch 00009: early stopping
Test Accuracy: 91.29999876022339
Done!
Epoch 1/100
180/180 [==============================] - 36s 178ms/step - loss: 0.6405 - accuracy: 0.6641 - val_loss: 0.2221 - val_accuracy: 0.9080
Epoch 2/100
180/180 [==============================] - 32s 177ms/step - loss: 0.1706 - accuracy: 0.9381 - val_loss: 0.1921 - val_accuracy: 0.9270
Epoch 3/100
180/180 [==============================] - 33s 182ms/step - loss: 0.0588 - accuracy: 0.9820 - val_loss: 0.2478 - val_accuracy: 0.9070
E

Epoch 6/100
180/180 [==============================] - 33s 186ms/step - loss: 0.0071 - accuracy: 0.9984 - val_loss: 0.6052 - val_accuracy: 0.9010
Epoch 7/100
180/180 [==============================] - 33s 185ms/step - loss: 0.0135 - accuracy: 0.9956 - val_loss: 0.3781 - val_accuracy: 0.9140
Epoch 8/100
180/180 [==============================] - 33s 185ms/step - loss: 0.0068 - accuracy: 0.9979 - val_loss: 0.3966 - val_accuracy: 0.9150
Restoring model weights from the end of the best epoch.
Epoch 00008: early stopping
Test Accuracy: 92.10000038146973
Done!

  Activation Filters       acc1       acc2       acc3       acc4       acc5  \
0       relu       1  92.199999  91.299999  92.400002  92.199999  92.400002   
1       relu       2  91.900003  91.799998  91.200000  92.400002  92.900002   
2       relu       3  92.900002  93.900001  92.400002  91.399997  92.500001   
3       relu       4  92.699999  93.599999  91.799998  92.000002  92.000002   
4       relu       5  91.299999  91.700000 

## Summary

In [37]:
record3.sort_values(by='AVG', ascending=False)

,Activation,Filters,acc1,acc2,acc3,acc4,acc5,acc6,acc7,acc8,acc9,acc10,AVG
2,relu,3,92.900002,93.900001,92.400002,91.399997,92.500001,92.199999,92.000002,92.600000,92.100000,92.000002,92.400001
0,relu,1,92.199999,91.299999,92.400002,92.199999,92.400002,92.199999,91.799998,91.700000,92.100000,92.199999,92.050000
3,relu,4,92.699999,93.599999,91.799998,92.000002,92.000002,91.500002,90.600002,91.100001,91.700000,93.400002,92.040001
1,relu,2,91.900003,91.799998,91.200000,92.400002,92.900002,91.200000,92.299998,91.299999,93.300003,91.399997,91.970000
4,relu,5,91.299999,91.700000,93.099999,92.900002,91.799998,92.500001,91.299999,89.499998,93.000001,91.900003,91.900000
5,relu,6,92.500001,91.799998,90.499997,91.900003,91.299999,92.699999,91.299999,92.799997,91.700000,92.100000,91.859999


In [38]:
report = record3.sort_values(by='AVG', ascending=False)
report = report.to_excel('TCN_SUBJ_3.xlsx', sheet_name='dynamic')